# TODO
- ~~figure out the right mask shape~~
- ~~figure out how to use distinct mask during training~~
- create iterative generate function
- make weird training loop & mechanism such that base model uses its inner models outputs instead of the correct output

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import time
from typing import List
import math
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
device = 'cpu'

In [27]:
# hyperparameters
b = 4 # how many independent sequences will we process in parallel?
t = 16 # what is the maximum context length for predictions?
max_iters = 10
eval_interval = 2
lr = 3e-4 # learning rate for each backprop step
eval_iters = 20
h = 4 # number of attention heads
l = 4 # number of transormer layers
dropout = 0.1 # % of parameters to ignore every iteration
l2 = 0.01 # multiplier for our L2 norm to encourage sparsity

# embedding aka hidden dimension. this is the largest that the model will have
d = 64 # make sure it is a power of 2
power_of_d = int(math.log2(d))

# the smallest power of 2 we'll be considering as a matryoshka embedding
min_power = 4 # Starting from 2^min_power
nesting_list = [2**i for i in range(min_power, int(power_of_d) + 1)]
print("embedding sizes: ", nesting_list)
print("number of nesting doll models: ", len(nesting_list), " (I will frequently refer to this number as 'g')")
toks_to_pred = [nesting_list[-1] // i for i in nesting_list]
print(f"the number of tokens we'll actually use from each model for one run of the main model: {toks_to_pred}")

embedding sizes:  [16, 32, 64]
number of nesting doll models:  3  (I will frequently refer to this number as 'g')
the number of tokens we'll actually use from each model for one run of the main model: [4, 2, 1]


In [4]:
# the dataset we'll be using is just TinyShakespeare
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print(text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [6]:
# here are all the unique characters that occur in this text. we'll do character-wise tokenization
chars = sorted(list(set(text)))
v = len(chars)
print(chars, v)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'] 65


In [7]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [8]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest validation
train_data = data[:n]
val_data = data[n:]

In [9]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - t, (b,))
    x = torch.stack([data[i:i+t] for i in ix])
    y = torch.stack([data[i+1:i+t+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [10]:
@torch.no_grad()
def estimate_loss(): # to use later during the training loop
    out = {}
    model.eval() # sets model to eval mode
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train() # just resets to training mode
    return out

In [11]:
class matryoshkaFeedFoward(nn.Module):
    def __init__(self, nesting_list: List, dropout):
        super().__init__()

        # our list of different potential embedding sizes
        self.nesting_list = nesting_list
        
        # the embedding dimension of the largest model
        self.d = nesting_list[-1]

        # Initialize only the largest weights and biases
        # this is more efficient than using regualr nn.Linear() because we need to splice them frequently
        self.w1 = nn.Parameter(torch.Tensor(self.d, 4 * self.d))
        self.b1 = nn.Parameter(torch.Tensor(4 * self.d))
        self.w2 = nn.Parameter(torch.Tensor(4 * self.d, self.d))
        self.b2 = nn.Parameter(torch.Tensor(self.d))

        # Initializing parameters
        nn.init.normal_(self.w1, std=0.02)  
        nn.init.normal_(self.b1, std=0.02)
        nn.init.normal_(self.w2, std=0.02)
        nn.init.normal_(self.b2, std=0.02)
        
        # the other parts
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(dropout)
        # notice how in forwardTuple(), the dropout mechanism will actually randomly drop out different weights for
        # each model size during a given forward pass. My intuition says this will actually be good for generalizability
        # but I suppose the opposite could be true. It'd be interesting to create a custom dropout method that ensures 
        # consistency in what parameters are dropped out across model sizes, but that's really not worth the effort
    
    def forwardTuple(self, x):
        """
        input: tuple of length g with tensors of shape (b,t,d_i) for d_i in nesting_list
        operation: 2 linear layers with a 4-times larger hidden depth, a relu nonlinearity in between, and then a dropout
        output: tuple of length g with tensors of shape (b,t,d_i) for d_i in nesting_list
        """
        out = ()
        for i, d_i in enumerate(self.nesting_list):
            out += (self.drop(self.relu(x[i] @ self.w1[:d_i,:4*d_i] + self.b1[:4*d_i]) @ self.w2[:4*d_i,:d_i] + self.b2[:d_i]),)
        return out

    def forwardTensor(self, x):
        """
        input: tensor of shape (b,t,d_i)
        operation: 2 linear layers with a 4-times depth, a relu nonlinearity in between, and then a dropout
        output: tensor of shape (b,t,d_i)
        """
        d_i = x.shape[-1]
        return self.relu(x @ self.w1[:d_i, :4*d_i] + self.b1[:4*d_i]) @ self.w2[:4*d_i, :d_i] + self.b2[:d_i]
    
    def forward(self, x):
        # forwardTuple() is for training and forwardTensor() is for inference
        # that will remain true for the rest of the code as well
        return self.forwardTuple(x) if type(x) == tuple else self.forwardTensor(x)

# ATTENTION

so in a normal NTP mask we'd just use lower-tril
```
[[1,0,0,0,0],
[1,1,0,0,0],
[1,1,1,0,0],
[1,1,1,1,0],
[1,1,1,1,1]]
```
but here we want the model to have the option of attending to a couple tokens into the future, but excluding the token that's its job to predict
```
[[1,0,1,0,0],
[1,1,0,1,0],
[1,1,1,0,1],
[1,1,1,1,0],
[1,1,1,1,1]]
```
how many? i'm thinking that the smallest matryoshka model should have to deal with regular NTP while each larger model should get to use the smallest's prediction for its own context. for now it'll probably be pretty inaccurate but i think later when i do this with FractalFormer i'll use each version to create multiple options for the model to choose from in batch

In [12]:
def create_extended_attention_mask(seq_length, future_tokens):
    tril = torch.tril(torch.ones((seq_length, seq_length)))
    
    # Instead of using loops, create a mask for future tokens
    # The diagonal of ones will be extended to the right by `future_tokens` positions
    if future_tokens > 0:
        mask = torch.tril(torch.ones((seq_length, seq_length)), diagonal=future_tokens+1) \
            -1*torch.tril(torch.ones((seq_length, seq_length)), diagonal=1) + tril
    else:
        # Create a standard lower triangular matrix using torch.tril
        mask = tril
        
    return mask

In [16]:
mask = create_extended_attention_mask(8, 2)
mask

tensor([[1., 0., 1., 1., 0., 0., 0., 0.],
        [1., 1., 0., 1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0., 1., 1., 0.],
        [1., 1., 1., 1., 1., 0., 1., 1.],
        [1., 1., 1., 1., 1., 1., 0., 1.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [17]:
class matryoshkaHead(nn.Module):
    def __init__(self, nesting_list: List, head_sizes: List):
        super().__init__()

        # to be used for iterating in forward()
        self.nesting_list = nesting_list
        self.head_sizes = head_sizes
        
        # the largest embedding dimension of the model
        self.d = nesting_list[-1]
        # the largest head size
        self.h = head_sizes[-1]

        # initialize only the largest. we'll subset later during forward()
        self.key = nn.Parameter(torch.Tensor(self.d, self.h)).to(device)
        self.query = nn.Parameter(torch.Tensor(self.d, self.h)).to(device)
        self.value = nn.Parameter(torch.Tensor(self.d, self.h)).to(device)
        
        # Initializing parameters
        nn.init.normal_(self.key, std=0.02)  
        nn.init.normal_(self.query, std=0.02)
        nn.init.normal_(self.value, std=0.02)

        # the mask so they only look into the past
        # the register_buffer('name', ...) ensures that a gradient is not kept & values are not updated
        #self.register_buffer('tril', torch.tril(torch.ones(t, t)))
        for i, h_i in enumerate(head_sizes):
            self.register_buffer(f'tril_{h_i}', create_extended_attention_mask(t, i))

    def forwardTuple(self, x):
        """
        input: tuple length g with tensors of shape (b,t,d_i) for d_i in nesting_list
        operation: 
            - weirdly masked self-attention. the smallest model has a regular NTP mask while the larger models get to see into the future
        output: tuple length g with tensors of shape (b,t,h_i) for h_i in head_sizes where h_i = d_i / h
        """
        k,q,v,wei,out = (),(),(),[],() # wei is a list so i can edit it in-place
        for i, (d_i, h_i) in enumerate(zip(self.nesting_list, self.head_sizes)):
            k += (x[i] @ self.key[:d_i, :h_i],)
            q += (x[i] @ self.query[:d_i, :h_i],)
            v += (x[i] @ self.value[:d_i, :h_i],)

            wei.append(q[i] @ k[i].transpose(-2,-1) * h_i ** -0.5) # k[i].shape[-1]**-0.5)
            wei[i] = wei[i].masked_fill(getattr(self, f'tril_{h_i}')[:t,:t] == 0, float('-inf')) # i think the [:t,:t] is redundant
            wei[i] = F.softmax(wei[i],dim=-1)
            
            out += (wei[i]@v[i],)
        return out

    def forwardTensor(self, x, h):
        """
        input: 
            - tensor of shape (b,t,d_i)
            - number of heads h
        operation: 
            - weirdly masked self-attention. the smallest model has a regular NTP mask while the larger models get to see into the future. 
            the tokens we'll be feeding the larger models come from the output of the smaller models. 
        output: tensor of shape (b,t,h_i) where h_i = d_i / h
        """
        d_i = x.shape[-1]
        h_i = d_i // h # the second / ensures it's an int rather than a float

        k = x @ self.key[:d_i, :h_i]
        q = x @ self.query[:d_i, :h_i]
        v = x @ self.value[:d_i, :h_i]

        wei = q @ k.transpose(-2,-1) * h_i ** -0.5 # k.shape[-1]**-0.5
        wei = wei.masked_fill(getattr(self, f'tril_{h_i}')[:t,:t] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)

        return wei @ v
        
    def forward(self, x, h=None):
        return self.forwardTuple(x) if type(x) == tuple else self.forwardTensor(x, h)

In [18]:
class matryoshkaMultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, h, nesting_list: List, head_sizes: List, dropout):
        super().__init__()
        
        self.nesting_list = nesting_list
        self.head_sizes = head_sizes
        self.h_count = h # number of heads
        self.d_count = len(nesting_list) # number of nesting doll sizes
        self.h_max = head_sizes[-1] # size of largest head
        self.d_max = nesting_list[-1] # size of largest embedding
        
        # creating all of our different attention heads, then storing them in a list for use later
        self.headsList = nn.ModuleList([matryoshkaHead(self.nesting_list, self.head_sizes) for _ in range(self.h_count)])
        
        # the linear projection that combines the outputs of all the heads
        self.weight = nn.Parameter(torch.Tensor(self.h_max * self.h_count, self.d_max)).to(device)
        self.bias = nn.Parameter(torch.Tensor(self.h_max * self.h_count)).to(device)
        
        # Initializing parameters
        nn.init.normal_(self.weight, std=0.02)  
        nn.init.normal_(self.bias, std=0.02)
        
        self.dropout = nn.Dropout(dropout)

    def forwardTuple(self, x):
        """
        input: tuple of length g with tensors of shape (b,t,d_i) for d_i=nesting_list[i]
        operation: 
            - perform self-attention w each head
                - input to each head: tuple of length g with tensors of shape (b,t,d_i) for d_i=nesting_list[i]
                - output from each head: tuple length g with tensors of shape (b,t,h_i) for h_i=head_sizes[i] where h_i = d_i / h
            - then concatenate into tuple of length g with tensors of shape (b,t,h*h_i). here by design h*h_i=d_i but it need not be that way
            - then linearly project each of the g tensors in the tuple back to (b,t,d_i)
        output: tuple of length g with tensors of shape (b,t,d_i) for d_i=nesting_list[i]
        """
        # let's get the outputs of each attention head
        # list length h of tuples length g of tensors shape (b,t,h_i) for h_i=d_i/h where d_i = nesting_list[i]
        head_outputs = [head(x) for head in self.headsList]

        # now let's reformat our ugly list of tuples into our usual expected tuple length g containing tensors shape (b,t,d_i)
        mid = ()
        for i in range(self.d_count):
            level = [] # where will store the output of each head for this model size
            for j, head in enumerate(head_outputs):
                level.append(head[i]) # this head's output for the d_i layer of the model
            
            # appending the concatenation of all the heads for this d_i layer of the model
            mid += (torch.cat(level, dim=-1),) # tuple length g with tensors of shape (b,t,d_i) for d_i=nesting_list[i]
        # mid is now a length g tuple of tensors shape (b,t,h*h_i)
        
        # now let's do our linear projection, which is not similar to how we did the matryoshkaFeedForward()
        # because we can't just select nested matrices within the primary matrix, we also have to account for the head 
        # concatenation which means skipping throughout and grabbing specific parts from the projection that match up
        #
        # so along the vertical of the matrix we want to iterate through self.nesting_list 
        # and along the horizontal we need to make skips the size of self.h
        # and then from those skips as starting points iteratively slice using self.head_sizes
        # then we concatenate those multiple spliced pieces along the horizontal
        # then we multiply a given output level by its respective projection
        out = ()
        for i, (d_i, h_i) in enumerate(zip(self.nesting_list, self.head_sizes)):
            # h_i is the head size of this iteration
            # j*self.h_max is our skip length
            this_levels_proj_w = torch.cat([self.weight[j*self.h_max:j*self.h_max+h_i, :d_i] for j in range(self.h_count)], dim=0)

            # bias is only one dimension so a bit simpler
            this_levels_proj_b = torch.cat([self.bias[j*self.h_max:j*self.h_max+h_i] for j in range(self.h_count)])

            # select correct level & multiply by weights then add bias
            # and can't forget to dropout
            out += (self.dropout(mid[i]@this_levels_proj_w + this_levels_proj_b),)
            
        return out

    def forwardTensor(self, x):
        """
        input: tensor of shape (b,t,d_i)
        operation: 
            - perform self-attention w each head
                - input to each head: tensor of shape (b,t,d_i)
                - output from each head: tensor of shape (b,t,h_i) where h_i = d_i / h
            - then concatenate the head outputs
            - then linearly project
        output: tensor of shape (b,t,d_i) 
        """
        d_i = x.shape[-1]
        h_i = d_i // self.h_count
        
        head_outputs = torch.cat([head(x, h=self.h_count) for head in self.headsList], dim=-1) # (b,t,h*h_i)

        spliced_projection_w = torch.cat([self.weight[j*self.h_max:j*self.h_max+h_i,:d_i] for j in range(self.h_count)], dim=0)
        spliced_projection_b = torch.cat([self.bias[j*self.h_max:j*self.h_max+h_i] for j in range(self.h_count)])

        return head_outputs @ spliced_projection_w + spliced_projection_b
        
    def forward(self, x):
        return self.forwardTuple(x) if type(x) == tuple else self.forwardTensor(x)

# LAYERNORM

Layernorm is relatively simple code-wise. However, of note is the fact that during training, the entire full length vector gets normalized whereas during inference we only layernorm the sub-vector we've been given if we're not using the full model size. This probably isn't a big deal since the sub-vectors are still hopefully being drawn from the same distribution during training. However, it wouldn't be surprising if the logits going into the small vectors are characteristically different from the full super-vectors, in which case this certainly might be a difficulty for the model. It might be worth changing this algorithm such that during training sub-vectors get normalized first and then held constant while super-vectors are normalized. something to think about. 

In [19]:
class matryoshkaLayerNorm(nn.Module):
    def __init__(self, nesting_list: List):
        super().__init__()

        self.nesting_list = nesting_list
        self.d_count = len(nesting_list)

        # we need layernorm attributes for each dimension size
        for d_i in nesting_list:
            setattr(self, f"ln_{d_i}", nn.LayerNorm(d_i, elementwise_affine=False))
            # we do elementwise_affine=False to remove the linear projection at the end
            # the linear projection would be counterproductive since we're layernorming in so many different places
        
    def forward(self, x):
        """
        a layernorm module that is dynamic to the input of either a single tensor or a tuple of tensors
        only works if the dimensions in question are in self.nesting_list

        input: either 
        - a tensor with last dimension equal to some value in self.nesting_list
        - a tuple of tensors where the last dimensions of each matches the values in self.nesting_list IN ORDER

        output: either of the above, but normalized
        """
        if type(x) == tuple:
            out = ()
            for i, d_i in enumerate(self.nesting_list):
                out += (getattr(self, f"ln_{d_i}")(x[i]),)
        else:
            d_i = x.shape[-1]
            out = getattr(self, f"ln_{d_i}")(x)
            
        return out

# RESIDUAL BLOCK

not a whole lot to say here other than the fact that i've chosen to pass everything through in the form of a tuple means that this block structure is HELLA inefficient in terms of memory. that's like 6 different copies of the tensors being forced to stay in memory goddamn

In [20]:
class matryoshkaBlock(nn.Module):
    def __init__(self, h, nesting_list: List, dropout):
        super().__init__()
        
        self.nesting_list = nesting_list
        self.head_sizes = [d_i // h for d_i in nesting_list]
        
        self.ln = matryoshkaLayerNorm(nesting_list)
        self.mha = matryoshkaMultiHeadAttention(h, nesting_list, self.head_sizes, dropout) 
        self.ffwd = matryoshkaFeedFoward(nesting_list, dropout)
    
    def forwardTuple(self, x_i):
        """
        input: length g tuple of shape (b,t,d_i) tensors for d_i in nesting_list
        output: length g tuple of shape (b,t,d_i) tensors for d_i in nesting_list
        """
        # please forgive my weird variable naming scheme

        # layernorming the input
        x_iplus1quart = self.ln(x_i)

        # the full multi-head attention
        attn = self.mha(x_iplus1quart)

        # residual connection for every residual state in our list of models
        x_iplus1half = tuple(x_i[j] + attn[j] for j in range(len(self.nesting_list)))

        # another layernorm
        x_iplus3quart = self.ln(x_iplus1half)

        # the feeforward
        ffwd = self.ffwd(x_iplus3quart)

        # the next residual connection for every residual state in our list of models
        x_iplus1 = tuple(x_iplus1half[j] + ffwd[j] for j in range(len(self.nesting_list)))
            
        return x_iplus1

    def forwardTensor(self, x):
        """
        input: tensor of shape (b,t,d_i)
        output: tensor of shape (b,t,d_i)
        """
        return x + self.ffwd(self.ln(x + self.mha(self.ln(x))))
        
    def forward(self, x):
        return self.forwardTuple(x) if type(x) == tuple else self.forwardTensor(x)

# OUTPUT

this output layer is similar to what you'll find in in [the original paper](https://arxiv.org/abs/2205.13147) except 
1) i use one output matrix instead of multiple
2) that output matrix i use is the transposed token embedding matrix
3) i add the option to perform inference rather than just training, which is something they did do in the [matformer paper](https://arxiv.org/pdf/2310.07707.pdf)

and then the loss function is the exact same

In [21]:
class matryoshkaOutputLayer(nn.Module):
    def __init__(self, embedding, nesting_list: List, num_classes):
        super().__init__()
        self.nesting_list = nesting_list
        self.num_classes = num_classes  # Number of tokens in the vocabulary
        
        self.embedding = embedding  # Store reference to the embedding matrix

        self.norm = matryoshkaLayerNorm(nesting_list)

    def forwardTuple(self, x):
        """
        input: length g tuple of tensors shape (b,t,d_i) for d_i in nesting_list
        operation: layernorm then multiply the final residual state by the transposed embedding matrix to get final logits
        output: length g tuple of tensors shape (b,t,v) where v is token vocabulary length
        """
        normed_logits = self.norm(x)
        normed_embeddings = self.norm(self.embedding).t()
        
        out = ()
        for i, d_i in enumerate(self.nesting_list):
            out += (normed_logits[i] @ normed_embeddings[:d_i,:],) 
            
        return out

    def forwardTensor(self, x):
        """
        input: tensor shape (b,t,d_i)
        operation: layernorm then multiply the final residual state by the transposed embedding matrix to get final logits
        output: tensor shape (b,t,v) where v is token vocabulary length
        """
        d_i = x.shape[-1]
        normed_logits = self.norm(x)
        normed_embeddings = self.norm(self.embedding[:,:d_i]).t()
        return normed_logits @ normed_embeddings
        
    def forward(self, x):
        return self.forwardTuple(x) if type(x) == tuple else self.forwardTensor(x)

In [22]:
class matryoshkaCEL(nn.Module):
    '''
    Loss function for Matryoshka Representation Learning
    we don't need to create a tensor version of the loss function bc training always involves all nesting levels
    '''
    def __init__(self):
        super().__init__()

        # the loss function
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, logits, target):
        """
        input: 
            - logits are a length g tuple each of shape [b batch size, t sequence length, v number of classes]
            - target is a shape [b batch size, t sequence length] tensor of the indices of the correct tokens
        output: a tensor containing a single float
        """
        g = len(logits)
        b,t,v = logits[0].shape

        # Calculate losses for each output and stack them
        losses = torch.stack([self.criterion(logits_i.view(b*t, v), target.view(b*t)) for logits_i in logits])

        return losses.sum()

# THE MODEL

In [ ]:
class matryoshkaGPT(nn.Module):
    def __init__(self, nesting_list: List, v, t, h, dropout):
        super().__init__()

        # the list of dimensions we'll be using
        self.nesting_list = nesting_list
        
        # the embedding size of the largest model
        self.d = nesting_list[-1]
        
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(v, self.d).to(device)
        
        # simple learned positional encodings rather than sine or RoPE
        # at some point i'm gonna write up a new model using all the stuff like RoPE that I should be using
        self.position_embedding_table = nn.Embedding(t, self.d).to(device)
        self.context_len = t

        # our special implementation of layernorm
        self.ln = matryoshkaLayerNorm(nesting_list)

        # bulk of the beast
        self.blocks = nn.Sequential(*[matryoshkaBlock(h, nesting_list, dropout) for _ in range(l)]) 

        # MATRYOSHKA OUTPUT HEADS
        self.out_heads = matryoshkaOutputLayer(self.token_embedding_table.weight, nesting_list, num_classes=v)
        
        # MATRYOSHKA LOSS
        self.loss = matryoshkaCEL()

    def forward(self, idx, targets=None, desired_d=nesting_list[-1]): 
        # desired_d is the desired dimension to use when performing inference (not used during training)

        # idx and targets are both (b,t) tensor of integers
        b, t = idx.shape
        
        pos_emb = self.position_embedding_table(torch.arange(t, device=device)) # (t,d)
        tok_emb = self.token_embedding_table(idx) # (b,t,d)
    
        if targets is None: 
            # if we are NOT training AKA just performing inference
            # send in a single matrix using desired_d
            x_0 = self.ln(tok_emb[:,:,:desired_d]) + pos_emb[:,:desired_d] # (b,t,d) + (t,d) -> (b,t,d)
        else:
            # if we ARE training
            # create tuple of residual states & send it thru
            x_0 = ()
            for d_i in self.nesting_list:
                x_0 += (self.ln(tok_emb[:,:,:d_i]) + pos_emb[:,:d_i],)
            # so in total the for loop gives us (b,t,d) & (t,d) -> g*(b,t,d_i) for d_i in nesting_list

        # most of the model is here
        x_f = self.blocks(x_0)

        # Matryoshka output head
        # self.out_heads includes within it the final layernorm
        logits = self.out_heads(x_f)

        loss = None if targets is None else self.loss(logits, targets) # g*(b,t,d) & (b,t) -> float

        return logits, loss

    def generateNTP(self, idx, max_new_tokens=100):
        """
        input: 
            - idx is (b, ?) tensor of indices from the current context
            - max_new_tokens sets generation length
        output: idx is (b,?+max_new_tokens) tensor of indices
        """
        
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -self.context_len:]
            
            # get the next prediction of the correct model
            logits, loss = self(idx_cond, desired_d=nesting_list[-1])

            # focus on the last timestep
            logits = logits[:, -1, :]
                    
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (b, d)

            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (b, 1)

            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (b, t+1)
            
        return idx

    def generateFast(self, idx, max_new_tokens=100):
        idx_temp = idx

        for i in range(model_count-1):
            for _ in range(i+1):
                for j in range(i+1):
                    # perform inference using the correct model as specified with j
                    logits, loss = self(idx_temp[:, -self.context_len:], desired_d=nesting_list[j])
        
                    # select the timestep of interest
                    logits = logits[:, replacement_index(j), :]
                            
                    # apply softmax to get probabilities
                    probs = F.softmax(logits, dim=-1) # (b, d)
                    
                    # sample from the distribution
                    idx_s_next = torch.multinomial(probs, num_samples=1) # (b, 1)
        
                    if j == 0:
                        # add the token to the sequence
                        idx_temp = torch.cat((idx_temp[:, -self.context_len:], idx_next), dim=1)
                    else:
                        # replace the previously predicted token
                        idx_temp[replacement_index(i)] = idx_m_next
        
        for i in range(max_new_tokens):
            for j in range(model_count):
                # perform inference using the correct model as specified with j
                logits, loss = self(idx_temp[:, -self.context_len:], desired_d=nesting_list[j])
        
                # select the timestep of interest
                logits = logits[:, replacement_index(j), :]
                            
                # apply softmax to get probabilities
                probs = F.softmax(logits, dim=-1) # (b, d)
                    
                # sample from the distribution
                idx_s_next = torch.multinomial(probs, num_samples=1) # (b, 1)
        
                if j == 0:
                    # add the token to the sequence
                    idx_temp = torch.cat((idx_temp[:, -self.context_len:], idx_next), dim=1)
                else:
                    # replace the previously predicted token
                    idx_temp[replacement_index(i)] = idx_m_next
                        
                    if i == len(self.nesting_list)-1:
                        # append the largest model's output to the actual sequence idx
                        idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [100]:
def replacement_index(n):
    return -1 -(n * (n + 1) // 2)
replacement_index(0)

-1

[0, \<where it changes\> 0,1,0,1,0,1,0,1,0,1,...]

[0,0,1,0,1, \<where it changes\> 0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,...]

[0,0,1,0,1,0,1,2,0,1,2,0,1,2, \<where it changes\> 0,1,2,3,0,1,2,3,0,1,2,3,0,1,2,3,0,1,2,3,4,...]

In [99]:
model_count = 5
max_new_tokens = 5

for i in range(model_count-1):
    for _ in range(i+1):
        for j in range(i+1):
            print(j)

print("\n")
for i in range(max_new_tokens):
    for j in range(model_count):
        print(j)

0
0
1
0
1
0
1
2
0
1
2
0
1
2
0
1
2
3
0
1
2
3
0
1
2
3
0
1
2
3


0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4


In [ ]:
### FAST

idx_temp = idx

for i in range(model_count-1):
    for _ in range(i+1):
        for j in range(i+1):
            print(j)

            # perform inference using the correct model as specified with j
            logits, loss = self(idx_temp[:, -self.context_len:], desired_d=nesting_list[j])

            # select the timestep of interest
            logits = logits[:, replacement_index(j), :]
                    
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (b, d)
            
            # sample from the distribution
            idx_s_next = torch.multinomial(probs, num_samples=1) # (b, 1)

            if j == 0:
                # add the token to the sequence
                idx_temp = torch.cat((idx_temp[:, -self.context_len:], idx_next), dim=1)
            else:
                # replace the previously predicted token
                idx_temp[replacement_index(i)] = idx_m_next

for i in range(max_new_tokens):
    for j in range(model_count):
        print(j)

            # perform inference using the correct model as specified with j
            logits, loss = self(idx_temp[:, -self.context_len:], desired_d=nesting_list[j])

            # select the timestep of interest
            logits = logits[:, replacement_index(j), :]
                    
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (b, d)
            
            # sample from the distribution
            idx_s_next = torch.multinomial(probs, num_samples=1) # (b, 1)

            if j == 0:
                # add the token to the sequence
                idx_temp = torch.cat((idx_temp[:, -self.context_len:], idx_next), dim=1)
            else:
                # replace the previously predicted token
                idx_temp[replacement_index(i)] = idx_m_next
                
                if i == len(self.nesting_list)-1:
                    # append the largest model's output to the actual sequence idx
                    idx = torch.cat((idx, idx_next), dim=1)







idx_temp = idx

### get the next prediction from the smallest model
logits, loss = self(idx_temp[:, -self.context_len:], desired_d=nesting_list[0])

# focus on the last timestep
logits = logits[:, -1, :]
                    
# apply softmax to get probabilities
probs = F.softmax(logits, dim=-1) # (b, d)

# sample from the distribution
idx_s_next = torch.multinomial(probs, num_samples=1) # (b, 1)

# add the token to the sequence
idx_temp = torch.cat((idx_temp, idx_s_next), dim=1)

for _ in range(2)
    ### get the next prediction from the smallest model
    logits, loss = self(idx_temp[:, -self.context_len:], desired_d=nesting_list[0])
    
    # focus on the last timestep
    logits = logits[:, -1, :]
                        
    # apply softmax to get probabilities
    probs = F.softmax(logits, dim=-1) # (b, d)
    
    # sample from the distribution
    idx_s_next = torch.multinomial(probs, num_samples=1) # (b, 1)
    
    # add the token to the sequence
    idx_temp = torch.cat((idx_temp, idx_s_next), dim=1)
    
    ### get the prediction from the medium model
    logits, loss = self(idx_temp[:, -self.context_len:], desired_d=nesting_list[1])
    
    # focus on the second-to-last timestep
    logits = logits[:, -2, :]
                        
    # apply softmax to get probabilities
    probs = F.softmax(logits, dim=-1) # (b, d)
    
    # sample from the distribution
    idx_m_next = torch.multinomial(probs, num_samples=1) # (b, 1)
    
    # replace the previously predicted token
    idx_temp[-2] = idx_m_next

for _ in range(max_new_tokens):
    for i in range(len(self.nesting_list)):
        
        ### get the prediction from the given model size. 0=small, 1=medium, 2=large, 3=xl
        logits, loss = self(idx_temp[:, -self.context_len:], desired_d=nesting_list[i])

        # focus on the correct timestep
        # i=0 for last timestamp, i=1 for 2nd to last(-1), i=2 for 4th to last(-2), i=3 for 7th to last(-3), etc
        logits = logits[:, replacement_index(i), :]
        
        # apply softmax to get probabilities
        probs = F.softmax(logits, dim=-1) # (b, d)
        
        # sample from the distribution
        idx_next = torch.multinomial(probs, num_samples=1) # (b, 1)

        if i == 0:
            # add the token to the sequence
            idx_temp = torch.cat((idx_temp[:, -self.context_len:], idx_next), dim=1)
        else:
            # replace the previously predicted token
            idx_temp[replacement_index(i)] = idx_m_next
            
            if i == len(self.nesting_list)-1:
                # append the largest model's output to the actual sequence idx
                idx = torch.cat((idx, idx_next), dim=1)
                










idx_temp = idx
# [c]

### get the next prediction from the smallest model
logits, loss = self(idx_temp[:, -self.context_len:], desired_d=nesting_list[0])

# focus on the last timestep
logits = logits[:, -1, :]
                    
# apply softmax to get probabilities
probs = F.softmax(logits, dim=-1) # (b, d)

# sample from the distribution
idx_s_next = torch.multinomial(probs, num_samples=1) # (b, 1)

# add the token to the sequence
idx_temp = torch.cat((idx_temp, idx_s_next), dim=1)
#[c,s]

### get the next prediction from the smallest model
logits, loss = self(idx_temp[:, -self.context_len:], desired_d=nesting_list[0])

# focus on the last timestep
logits = logits[:, -1, :]
                    
# apply softmax to get probabilities
probs = F.softmax(logits, dim=-1) # (b, d)

# sample from the distribution
idx_s_next = torch.multinomial(probs, num_samples=1) # (b, 1)

# add the token to the sequence
idx_temp = torch.cat((idx_temp, idx_s_next), dim=1)
#[c,s,s]

### get the prediction from the medium model
logits, loss = self(idx_temp[:, -self.context_len:], desired_d=nesting_list[1])

# focus on the second-to-last timestep
logits = logits[:, -2, :]
                    
# apply softmax to get probabilities
probs = F.softmax(logits, dim=-1) # (b, d)

# sample from the distribution
idx_m_next = torch.multinomial(probs, num_samples=1) # (b, 1)

# replace the previously predicted token
idx_temp[-2] = idx_m_next
#[c,m,s]

### get the next prediction from the smallest model
logits, loss = self(idx_temp[:, -self.context_len:], desired_d=nesting_list[0])

# focus on the last timestep
logits = logits[:, -1, :]
                    
# apply softmax to get probabilities
probs = F.softmax(logits, dim=-1) # (b, d)

# sample from the distribution
idx_s_next = torch.multinomial(probs, num_samples=1) # (b, 1)

# add the token to the sequence
idx_temp = torch.cat((idx_temp[:, -self.context_len:], idx_s_next), dim=1)
#[c,m,s,s]

### get the prediction from the medium model
logits, loss = self(idx_temp[:, -self.context_len:], desired_d=nesting_list[1])

# focus on the second-to-last timestep
logits = logits[:, -2, :]
                    
# apply softmax to get probabilities
probs = F.softmax(logits, dim=-1) # (b, d)

# sample from the distribution
idx_m_next = torch.multinomial(probs, num_samples=1) # (b, 1)

# replace the previously predicted token
idx_temp[-2] = idx_m_next
#[c,m,m,s]

### get the next prediction from the smallest model
logits, loss = self(idx_temp[:, -self.context_len:], desired_d=nesting_list[0])

# focus on the last timestep
logits = logits[:, -1, :]
                    
# apply softmax to get probabilities
probs = F.softmax(logits, dim=-1) # (b, d)

# sample from the distribution
idx_s_next = torch.multinomial(probs, num_samples=1) # (b, 1)

# add the token to the sequence
idx_temp = torch.cat((idx_temp[:, -self.context_len:], idx_s_next), dim=1)
#[c,m,m,s,s]

### get the prediction from the medium model
logits, loss = self(idx_temp[:, -self.context_len:], desired_d=nesting_list[1])

# focus on the second-to-last timestep
logits = logits[:, -2, :]
                    
# apply softmax to get probabilities
probs = F.softmax(logits, dim=-1) # (b, d)

# sample from the distribution
idx_m_next = torch.multinomial(probs, num_samples=1) # (b, 1)

# replace the previously predicted token
idx_temp[-2] = idx_m_next
#[c,m,m,m,s]

### get prediction from large model
logits, loss = self(idx_temp[:, -self.context_len:], desired_d=nesting_list[2])

# focus on the fourth-to-last timestep
logits = logits[:, -4, :]
                    
# apply softmax to get probabilities
probs = F.softmax(logits, dim=-1) # (b, d)

# sample from the distribution
idx_l_next = torch.multinomial(probs, num_samples=1) # (b, 1)

# replace the previously predicted token
idx_temp[-4] = idx_l_next
#[c,l,m,m,s]

idx = torch.cat((idx, idx_l_next), dim=1)

### get the next prediction from the smallest model
logits, loss = self(idx_temp[:, -self.context_len:], desired_d=nesting_list[0])

# focus on the last timestep
logits = logits[:, -1, :]
                    
# apply softmax to get probabilities
probs = F.softmax(logits, dim=-1) # (b, d)

# sample from the distribution
idx_s_next = torch.multinomial(probs, num_samples=1) # (b, 1)

# add the token to the sequence
idx_temp = torch.cat((idx_temp[:, -self.context_len:], idx_s_next), dim=1)
#[c,l,m,m,s,s]

### get the prediction from the medium model
logits, loss = self(idx_temp[:, -self.context_len:], desired_d=nesting_list[1])

# focus on the second-to-last timestep
logits = logits[:, -2, :]
                    
# apply softmax to get probabilities
probs = F.softmax(logits, dim=-1) # (b, d)

# sample from the distribution
idx_m_next = torch.multinomial(probs, num_samples=1) # (b, 1)

# replace the previously predicted token
idx_temp[-2] = idx_m_next
#[c,l,m,m,m,s]

### get prediction from large model
logits, loss = self(idx_temp[:, -self.context_len:], desired_d=nesting_list[2])

# focus on the fourth-to-last timestep
logits = logits[:, -4, :]
                    
# apply softmax to get probabilities
probs = F.softmax(logits, dim=-1) # (b, d)

# sample from the distribution
idx_l_next = torch.multinomial(probs, num_samples=1) # (b, 1)

# replace the previously predicted token
idx_temp[-4] = idx_l_next
#[c,l,l,m,m,s]

idx = torch.cat((idx, idx_l_next), dim=1)

### get the next prediction from the smallest model
logits, loss = self(idx_temp[:, -self.context_len:], desired_d=nesting_list[0])

# focus on the last timestep
logits = logits[:, -1, :]
                    
# apply softmax to get probabilities
probs = F.softmax(logits, dim=-1) # (b, d)

# sample from the distribution
idx_s_next = torch.multinomial(probs, num_samples=1) # (b, 1)

# add the token to the sequence
idx_temp = torch.cat((idx_temp[:, -self.context_len:], idx_s_next), dim=1)
#[c,l,l,m,m,s,s]

### get the prediction from the medium model
logits, loss = self(idx_temp[:, -self.context_len:], desired_d=nesting_list[1])

# focus on the second-to-last timestep
logits = logits[:, -2, :]
                    
# apply softmax to get probabilities
probs = F.softmax(logits, dim=-1) # (b, d)

# sample from the distribution
idx_m_next = torch.multinomial(probs, num_samples=1) # (b, 1)

# replace the previously predicted token
idx_temp[-2] = idx_m_next
#[c,l,l,m,m,m,s]

### get prediction from large model
logits, loss = self(idx_temp[:, -self.context_len:], desired_d=nesting_list[2])

# focus on the fourth-to-last timestep
logits = logits[:, -4, :]
                    
# apply softmax to get probabilities
probs = F.softmax(logits, dim=-1) # (b, d)

# sample from the distribution
idx_l_next = torch.multinomial(probs, num_samples=1) # (b, 1)

# replace the previously predicted token
idx_temp[-4] = idx_l_next
#[c,l,l,l,m,m,s]

idx = torch.cat((idx, idx_l_next), dim=1)

### get the next prediction from the smallest model
logits, loss = self(idx_temp[:, -self.context_len:], desired_d=nesting_list[0])

# focus on the last timestep
logits = logits[:, -1, :]
                    
# apply softmax to get probabilities
probs = F.softmax(logits, dim=-1) # (b, d)

# sample from the distribution
idx_s_next = torch.multinomial(probs, num_samples=1) # (b, 1)

# add the token to the sequence
idx_temp = torch.cat((idx_temp[:, -self.context_len:], idx_s_next), dim=1)
#[c,l,l,l,m,m,s,s]

### get the prediction from the medium model
logits, loss = self(idx_temp[:, -self.context_len:], desired_d=nesting_list[1])

# focus on the second-to-last timestep
logits = logits[:, -2, :]
                    
# apply softmax to get probabilities
probs = F.softmax(logits, dim=-1) # (b, d)

# sample from the distribution
idx_m_next = torch.multinomial(probs, num_samples=1) # (b, 1)

# replace the previously predicted token
idx_temp[-2] = idx_m_next
#[c,l,l,l,m,m,m,s]

### get prediction from large model
logits, loss = self(idx_temp[:, -self.context_len:], desired_d=nesting_list[2])

# focus on the fourth-to-last timestep
logits = logits[:, -4, :]
                    
# apply softmax to get probabilities
probs = F.softmax(logits, dim=-1) # (b, d)

# sample from the distribution
idx_l_next = torch.multinomial(probs, num_samples=1) # (b, 1)

# replace the previously predicted token
idx_temp[-4] = idx_l_next
#[c,l,l,l,l,m,m,s]

### get prediction from XL model
logits, loss = self(idx_temp[:, -self.context_len:], desired_d=nesting_list[3])

# focus on the seventh-to-last timestep
logits = logits[:, -7, :]
                    
# apply softmax to get probabilities
probs = F.softmax(logits, dim=-1) # (b, d)

# sample from the distribution
idx_xl_next = torch.multinomial(probs, num_samples=1) # (b, 1)

# replace the previously predicted token
idx_temp[-7] = idx_l_next
#[c,xl,l,l,l,m,m,s]


#[c,xl,xl,xl,xl,xl,l,l,l,m,m,s]



# append sampled index to the running sequence
idx = torch.cat((idx, idx_next), dim=1) # (b, t+1)



In [43]:
n=4
-1 -(n * (n + 1) // 2)



-11

# TRAINING

In [48]:
model = matryoshkaGPT(nesting_list, v, t, h, dropout).to(device)
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=l2)
# print the number of parameters in the model
print(sum(p.numel() for p in model.parameters())/1e3, 'K parameters')

203.328 K parameters


In [49]:
start_time = time.time()

# Enable anomaly detection
#torch.autograd.set_detect_anomaly(True)

for iter in range(max_iters):

    # sample a batch of data
    xb, yb = get_batch('train')
    
    # train
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        current_time = time.time()
        elapsed_time = current_time - start_time
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}, time elapsed: {elapsed_time:.2f} seconds")

# Disable anomaly detection after the training loop
#torch.autograd.set_detect_anomaly(False)

step 0: train loss 80.4312, val loss 79.9946, time elapsed: 0.12 seconds
step 2: train loss 79.0773, val loss 80.6407, time elapsed: 0.49 seconds
step 4: train loss 79.0374, val loss 78.9038, time elapsed: 0.85 seconds
step 6: train loss 77.9571, val loss 77.7309, time elapsed: 1.21 seconds
step 8: train loss 77.3744, val loss 77.1806, time elapsed: 1.58 seconds
step 9: train loss 77.0406, val loss 75.9463, time elapsed: 1.91 seconds


In [25]:
## save the trained model
torch.save(model.state_dict(), f'models/{model.__class__.__name__}_b{b}_t{t}_d{d}_h{h}_l{l}_lr{lr}_drop{dropout}_l2-{l2}_min_power{min_power}_{time.strftime("%Y-%m-%d|%H-%M-%S")}.pth')

# Load a saved model

In [ ]:
model = matryoshkaGPT().to(device)  # Initialize a model with the same architecture

# Load the saved state dictionary
model.load_state_dict(torch.load('models/matryoshkaGPT_b16_t64_d2_h4_l8_lr0.0003_drop0.1_l2-0.01_min_power5_2024-02-13|01-55-54.pth'))

# If you plan to continue training the model, switch to training mode
#model.train()

# If you only plan to do inference, switch to evaluation mode
model.eval()

# Inference

In [24]:
input_str = "JULIET:\nO Romeo, Romeo! wherefore art thou Romeo?\nDeny thy fathe" # the classic line
context_tensor = torch.tensor([encode(input_str)], dtype=torch.long, device=device)
for d in range(len(nesting_list)):
    print("-----------------model: ", d, "------------------")
    output = model.generate(context_tensor, max_new_tokens=100, degree=d) # -1 for biggest model size
    output_str = decode(output[0].tolist())
    print(output_str)

-----------------model:  0 ------------------
JULIET:
O Romeo, Romeo! wherefore art thou Romeo?
Deny thy fathe, d.

Th,
Cw.
BBUBBt,
Mur pwnd, ming wid wit?
Thifck,
Y:
Thow, wwive,
GUBun,
ICowid, IUCowe ITELOUUB
-----------------model:  1 ------------------
JULIET:
O Romeo, Romeo! wherefore art thou Romeo?
Deny thy fathend in ch cow aws hy;;;:
Sur tht chak'd wn.
Twior y--bixck- d lifurer nd mived,
ANGt,
O, craver hath,
-----------------model:  2 ------------------
JULIET:
O Romeo, Romeo! wherefore art thou Romeo?
Deny thy fathere thener preaks.
Be'd her:
Proute:
K.
War.
A nothinghts;

Give canggeme
Hirt tivesbe-d ZZZUp, JUpea


### obviously given the size of this model it's not very good. oh well
idk about you but it looks to me like the biggest model is the best, as you'd expect. it seems to have a better understanding of the length of a word. also these outputs would prolly be better if i scaled the logits with a temperature but it's late and i'm tired